# Loan Prediction
#### Building a model to predict if a loan will get approved or not. 
#### Classification problem. 
#### Using Logistic Regression (for now)

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib as plt

# loading the dataset
dataset = pd.read_csv('loans_data.csv')
dataset

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [55]:
# dealing with missing values if there are any

# filling in missing values with mean value of that variable/column
loan_amnt = dataset['LoanAmount']
loan_amnt.isnull().value_counts()
loan_amnt.fillna(loan_amnt.mean(), inplace=True)
loan_amnt.isnull().value_counts()

loan_amnt_trm = dataset['Loan_Amount_Term']
loan_amnt_trm.isnull().value_counts()
loan_amnt_trm.fillna(loan_amnt_trm.mean())

False    600
True      14
Name: Loan_Amount_Term, dtype: int64

In [15]:
# Categorical boolean mask
categorical_feature_mask = (cl_dataset.dtypes == object)

# filter categorical columns using mask and turn it into a list
categorical_cols = cl_dataset.columns[categorical_feature_mask].tolist()

### LabelEncoder converts each class under specified feature to a numerical value

In [21]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder

# instantiate labelencoder object
le = LabelEncoder()

# apply le on categorical feature columns
cl_dataset[categorical_cols] = cl_dataset[categorical_cols].apply(lambda col: le.fit_transform(col))
cl_dataset[categorical_cols]

/home/pako/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
1,0,1,1,1,0,0,0,0
2,1,1,1,0,0,1,2,1
3,2,1,1,0,1,0,2,1
4,3,1,0,0,0,0,2,1
5,4,1,1,2,0,1,2,1
...,...,...,...,...,...,...,...,...
609,475,0,0,0,0,0,0,1
610,476,1,1,3,0,0,0,1
611,477,1,1,1,0,0,2,1
612,478,1,1,2,0,0,2,1


In [19]:
y = cl_dataset[categorical_cols]['Loan_Status'].values
X = cl_dataset.iloc[:, [6, 7, 8, 9, 10]].values

# Splitting the data into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Feature scaling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# Fitting Logistic Regression to training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs')
classifier.fit(X_train, y_train)

# Predict Test set results
y_pred = classifier.predict(X_test)
y_pred

# Create the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[12, 24],
       [ 3, 81]])

In [18]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
print ("Model accuracy is: ", score)

Model accuracy is:  0.8166666666666667
